## Basic set

In [1]:
import os, sys, random, struct, argparse
import numpy as np
np.set_printoptions(precision=16)

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=6, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])

## Integer -> Two's Complemenet Binary

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

## XOR gate

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else :
        iOut = '0'
    return iOut

## Fixed

In [5]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [6]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol		= 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [7]:
class LFSR:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[2]),int(stream[3]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3

In [8]:
b = LFSR()
for i in range(16) :   
    if i == 0 :
        a = b.Random()
        print(a)
    if i == 15 :
        a = b.Allzero()
        print(a)
    else :
        a = b.Normal(a)
        print(a)

0100
0010
1001
1100
0110
1011
0101
1010
1101
1110
1111
0111
0011
0001
1000
0100
0000


## Comparator

In [9]:
def Comp(a,lfsr,snum):
    for com in range(0,len(lfsr)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

## SNUM

In [10]:
def snum(a):
    if a > 0 :
        return '0'
    else :
        return '1'

## SNG module

In [15]:
def SNG(iIN):

    sNUM = snum(iIN)
    
    bIN = flp2fix(a,args.full_bits,args.frac_bits).bFull
    oAlist = []
    #lfsrlist = []
    
    lfsr = LFSR()
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if k == 0:
            lNUM = lfsr.Random()
        else :
            lNUM = lfsr.Normal(lNUM)
        if (k == 2**(args.bBW)-1):
            lNUM = lfsr.Allzero()
        
        if len(lNUM) != len(bIN[-(args.frac_bits):]) :
            if len(lNUM) < len(bIN[-(args.frac_bits):]) :
                lNUM = lNUM + (len(bIN[-(args.frac_bits):])-len(lNUM))*'0'
            else :
                print("it can't work")
                return 0
        print(lNUM)
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
        
    oAnum = oAlist.count('1')
    
    print(iIN*16)
    print(oAnum)
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    return sA,oAnum

In [16]:
for i in range(10):
    a = random.random()
    b = flp2fix(a,args.full_bits,args.frac_bits).fFull
    c = flp2fix(a,args.full_bits,args.frac_bits).bFull
    d = flp2fix(b,args.full_bits,args.frac_bits).fFull
    
    e,f = SNG(b)
    
    print(f'random number is : {a}')
    print(f'changed fixed number is : {b}')
    print(f'binary fixed number is : {c[2:]}')
    print(f'stochastic number is : {e}')
    print(f'recovered fixed number is : {f/len(e)}')
    print(f'--------------------------------------------')

101000
110100
111000
111100
011100
001100
000100
100000
010000
001000
100100
110000
011000
101100
010100
000000
6.25
7
random number is : 0.40565993501503617
changed fixed number is : 0.390625
binary fixed number is : 011001
stochastic number is : 00000011011001011
recovered fixed number is : 0.4117647058823529
--------------------------------------------
111100
011100
001100
000100
100000
010000
001000
100100
110000
011000
101100
010100
101000
110100
111000
000000
6.0
6
random number is : 0.38824645933240354
changed fixed number is : 0.375
binary fixed number is : 011000
stochastic number is : 00011011000010001
recovered fixed number is : 0.35294117647058826
--------------------------------------------
100000
010000
001000
100100
110000
011000
101100
010100
101000
110100
111000
111100
011100
001100
000100
000000
4.25
5
random number is : 0.2736186272760659
changed fixed number is : 0.265625
binary fixed number is : 010001
stochastic number is : 00110000000000111
recovered fixed number